# Noisy GDBFアルゴリズムの実装例

* Noisy GDBFアルゴリズムは優れたビットフリップ型復号法です。詳細は下記の論文を参照してください。
https://ieeexplore.ieee.org/document/6894606

* ldpclib には、疎行列（検査行列）の読み込み、生成行列の生成が可能です。これらの関数は、名工大修了生の福本真也君の作です。
* 本プログラムで学習はしていません。

## 必要なパッケージをインポートする

In [29]:
import numpy as np
import torch
import torch.nn as nn
import math
import time
import ldpclib as l

## デバイスの指定

In [30]:
device = torch.device('cuda') # specify 'cpu' or 'cuda'

## グローバル定数の設定

In [28]:
filename = 'PEGReg504x1008.dec' # spmat 形式(alist形式と少しだけ違う)
H_spmat = l.read_spmat(filename)
H = H_spmat.to_ndarray() # 検査行列
G = l.to_generator_matrix(H) # 生成行列
(np.matmul(G, H.T) % 2 == 0).all() # check if G H^T = 0 or not
n = H.shape[1] # 符号長
m = H.shape[0] # 冗長
k = G.shape[0] # 情報記号長(次元)
dtype = torch.float
H = torch.from_numpy(H).float().to(device) # 検査行列
G = torch.from_numpy(G).float().to(device) # 生成行列

In [25]:
snr        = 4.5 # singal to noise ratio
rate       = 1.0 - m/n # coding rate
offset_val = 0.4 # noise offset
fluc_std   = math.sqrt(0.4)  # standard deviation of noise
num_itr    = 20    # number of iterations
max_loop   = 1000  # number of simulation loops 
bs         = 10 # minibatch size

## BF関連の関数

In [23]:
def bipolar_syndrome(x, H):
    return 1.0 - 2.0 * torch.mm(H, 0.5 * (1.0 - x.t())).fmod(2.0).t()
def syndrome_sum(s, H):
    return torch.mm(s, H)

## Noisy GDBF 復号器クラス

In [26]:
class GDBF(nn.Module):
    def __init__(self, offset_val, fluc_std):
        super(GDBF, self).__init__()
        self.fluc_std = fluc_std
        self.offset   = offset_val * torch.ones(bs, n).to(device)
    def forward(self, word, num_itr):        
        y = word + torch.normal(torch.zeros(bs, n), noise_std).to(device)
        x = torch.sign(y) 
        for i in range(num_itr):
            s = bipolar_syndrome(x, H)
            r = x * y + syndrome_sum(s, H)  + self.offset + torch.normal(torch.zeros(1, n), self.fluc_std).to(device)
            x =  x * torch.sign(r)
        return x

## シミュレーションループ

In [27]:
gdbf = GDBF(offset_val, fluc_std).to(device)
num_errs = 0
num_bits = 0
noise_std = math.sqrt((0.5 * pow(10.0, -snr/10.0))/rate)
start     = time.time()
print('snr  = ', snr)
print('rate = ', rate)
print('noise_std = ', noise_std)
for loop in range(max_loop):
    message = torch.bernoulli(0.5 * torch.ones(bs, k)).to(device)
    word = 1.0 - 2.0 * torch.mm(message, G).fmod(2.0)
    word = word.to(device)
    x_hat = gdbf(word, num_itr)
    num_errs += (x_hat != word).sum().detach().item()
    num_bits += n * bs
print('BER      = {:.4e}'.format(num_errs/num_bits))
print('num_bits = {:.4e}'.format(1.0*num_bits))
elapsed_time = time.time() - start
print('elapsed_time:{0}'.format(elapsed_time) + '(sec)')

snr  =  4.5
rate =  0.5
noise_std =  0.5956621435290105
BER      = 6.2183e-04
num_bits = 1.0080e+07
elapsed_time:14.283835411071777(sec)
